In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
def extract_street_name(address):
    parts = address.split()
    
    for part in reversed(parts):
        if not part.isnumeric():
            return part
    
    return None

address = "2911 86 STREET"
street_name = extract_street_name(address)

print(street_name)


STREET


In [4]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [5]:
df = w.wrangle_coll_stage1(year_to_retrieve, app_token)
df.head()

CSV file for 2022 already exists. Loading data from the CSV.


,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,4:30,4491626,NaN,NaN,NaN,Southern parkway,NaN,Jfk expressway,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,5:17,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,1:30,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.91799,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,520 EAST 137 STREET,NaN,BRONX,10454.0,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,BORDEN AVENUE,NaN,NaN,1,0,0,0,0,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified


In [7]:
missing_df = df[(df['latitude'].notnull()) & (df[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]


In [8]:
len(missing_df)

29408

In [17]:
df_geo = missing_df.head(50)
df_geo

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
1,2022-01-01 05:17:00,2022-01-01,5:17,4491857,40.746930,-73.848660,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,1:30,4491344,40.819157,-73.960380,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,520 EAST 137 STREET,NaN,BRONX,10454.0,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
6,2022-01-01 02:53:00,2022-01-01,2:53,4491586,40.646034,-73.996780,"{'latitude': '40.646034', 'longitude': '-73.99...",NaN,850 40 STREET,NaN,BROOKLYN,11232.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified
11,2022-01-01 17:00:00,2022-01-01,17:00,4491660,40.701195,-73.914090,"{'latitude': '40.701195', 'longitude': '-73.91...",WYCKOFF AVENUE,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Passing Too Closely,Sedan,Unspecified
13,2022-01-01 00:55:00,2022-01-01,0:55,4491201,40.673107,-73.999590,"{'latitude': '40.673107', 'longitude': '-73.99...",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Sedan,Unspecified
15,2022-01-01 12:22:00,2022-01-01,12:22,4491279,40.666134,-73.884920,"{'latitude': '40.666134', 'longitude': '-73.88...",LIVONIA AVENUE,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Sedan,Unspecified,Sedan,Unspecified
16,2022-01-01 12:21:00,2022-01-01,12:21,4491659,40.684937,-73.910140,"{'latitude': '40.684937', 'longitude': '-73.91...",BUSHWICK AVENUE,NaN,NaN,NaN,NaN,3,0,0,0,0,0,3,0,Sedan,Driver Inattention/Distraction,Sedan,Unspecified
18,2022-01-01 02:30:00,2022-01-01,2:30,4491670,40.806410,-73.950200,"{'latitude': '40.80641', 'longitude': '-73.950...",7 AVENUE,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Taxi,Traffic Control Disregarded,Taxi,Unspecified
21,2022-01-01 14:43:00,2022-01-01,14:43,4491406,40.769993,-73.915825,"{'latitude': '40.769993', 'longitude': '-73.91...",GRAND CENTRAL PKWY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,NaN


In [12]:
import pandas as pd
import requests
import time

def fill_missing_geocode_info(df, api_key, save_path):
    # Create a list to store geocode data
    geocode_data = []
    
    # Iterate through rows of the original DataFrame
    for index, row in df.iterrows():
        start_time = time.time()  # Record the start time for this iteration
        
        # Check if ZIP code, borough, and on-street name are missing
        if pd.isna(row['zip_code']) or pd.isna(row['borough']) or pd.isna(row['on_street_name']):
            lat = row['latitude']
            lng = row['longitude']
            
            # Make the API request
            url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                
                # Initialize variables to store the address components
                zipcode = None
                borough = None
                street_address = None
                
                # Parse the response to extract address components
                for result in data.get('results', []):
                    for component in result.get('address_components', []):
                        if 'postal_code' in component['types']:
                            zipcode = component['short_name']
                        if 'sublocality_level_1' in component['types']:
                            borough = component['long_name']
                        if 'route' in component['types']:
                            street_address = component['long_name']
                
                # Fill in missing information in the DataFrame
                if pd.isna(row['zip_code']):
                    df.at[index, 'zip_code'] = zipcode
                if pd.isna(row['borough']):
                    df.at[index, 'borough'] = borough
                if pd.isna(row['on_street_name']):
                    df.at[index, 'on_street_name'] = street_address
                
                # Append geocode data to the list
                geocode_data.append({'latitude': lat, 'longitude': lng, 'geocode_json': data})
            else:
                print(f"Error: {response.status_code} for row {index}")
        
        end_time = time.time()  # Record the end time for this iteration
        iteration_time = end_time - start_time
        print(f"Iteration {index}: {iteration_time} seconds")
    
    # Convert the list of dictionaries to a DataFrame
    geocode_df = pd.DataFrame(geocode_data)
    
    # Save the geocode data to a CSV file
    geocode_df.to_csv(save_path, index=False)
    
    return df



In [13]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geocode_data.csv'

# Fill missing geocode information in the DataFrame
filled_df = fill_missing_geocode_info(df_geo, api_key, save_path)

# Display the updated DataFrame
filled_df.to_csv('filled.csv')


Iteration 1: 0.2135002613067627 seconds
Iteration 2: 0.3295750617980957 seconds
Iteration 3: 5.1975250244140625e-05 seconds
Iteration 6: 2.6941299438476562e-05 seconds
Iteration 11: 0.23658204078674316 seconds
Iteration 13: 0.3198709487915039 seconds
Iteration 15: 0.2469620704650879 seconds
Iteration 16: 0.21641993522644043 seconds
Iteration 18: 0.24823808670043945 seconds
Iteration 21: 0.31407690048217773 seconds
Iteration 23: 0.19623231887817383 seconds
Iteration 24: 0.27939701080322266 seconds
Iteration 25: 0.2218170166015625 seconds
Iteration 29: 0.4270310401916504 seconds
Iteration 30: 0.24576067924499512 seconds
Iteration 34: 0.2953457832336426 seconds
Iteration 35: 0.304563045501709 seconds
Iteration 36: 0.17869329452514648 seconds
Iteration 38: 0.1861429214477539 seconds
Iteration 39: 0.2581498622894287 seconds
Iteration 40: 0.3642001152038574 seconds
Iteration 43: 0.2398509979248047 seconds
Iteration 44: 0.38193511962890625 seconds
Iteration 46: 0.1917729377746582 seconds
Iter

In [10]:
import pandas as pd
import requests

def fill_missing_geocode_info(df, api_key, save_path):
    # Create a list to store geocode data
    geocode_data = []
    
    # Iterate through rows of the original DataFrame
    for index, row in df.iterrows():
        # Check if ZIP code, borough, and on-street name are missing
        if pd.isna(row['zip_code']) or pd.isna(row['borough']) or pd.isna(row['on_street_name']):
            lat = row['latitude']
            lng = row['longitude']
            
            # Make the API request
            url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                
                # Initialize variables to store the address components
                zipcode = None
                borough = None
                street_address = None
                
                # Parse the response to extract address components
                for result in data.get('results', []):
                    for component in result.get('address_components', []):
                        if 'postal_code' in component['types']:
                            zipcode = component['short_name']
                        if 'sublocality_level_1' in component['types']:
                            borough = component['long_name']
                        if 'route' in component['types']:
                            street_address = component['long_name']
                
                # Fill in missing information in the DataFrame
                if pd.isna(row['zip_code']):
                    df.at[index, 'zip_code'] = zipcode
                if pd.isna(row['borough']):
                    df.at[index, 'borough'] = borough
                if pd.isna(row['on_street_name']):
                    df.at[index, 'on_street_name'] = street_address
                
                # Append geocode data to the list
                geocode_data.append({'latitude': lat, 'longitude': lng, 'geocode_json': data})
            else:
                print(f"Error: {response.status_code} for row {index}")
    
    # Convert the list of dictionaries to a DataFrame
    geocode_df = pd.DataFrame(geocode_data)
    
    # Save the geocode data to a CSV file
    geocode_df.to_csv(save_path, index=False)
    
    return df


In [ ]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geocode_data.csv'

# Fill missing geocode information in the DataFrame
filled_df = fill_missing_geocode_info(missing_df, api_key, save_path)

# Display the updated DataFrame
filled_df.to_csv('filled.csv')


In [17]:
filled_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
185,2022-01-02 00:00:00,2022-01-02,0:00,4492267,40.862392,-73.918020,"{'latitude': '40.862392', 'longitude': '-73.91...",WEST 205 STREET,NaN,NaN,Manhattan,10034,0,0,0,0,0,0,0,0,Sedan,Driver Inattention/Distraction,Sedan,Unspecified
191,2022-01-02 20:45:00,2022-01-02,20:45,4492469,40.835484,-73.949400,"{'latitude': '40.835484', 'longitude': '-73.94...",HENRY HUDSON PARKWAY,NaN,NaN,Manhattan,10032,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Reaction to Uninvolved Vehicle,Station Wagon/Sport Utility Vehicle,Unspecified
192,2022-01-02 09:50:00,2022-01-02,9:50,4492521,40.674000,-73.960396,"{'latitude': '40.674', 'longitude': '-73.96039...",CLASSON AVENUE,NaN,NaN,Brooklyn,11238,0,0,0,0,0,0,0,0,Sedan,Passing Too Closely,Sedan,Unspecified
194,2022-01-02 15:27:00,2022-01-02,15:27,4493113,40.681780,-73.981920,"{'latitude': '40.68178', 'longitude': '-73.981...",Warren Street,572 WARREN STREET,NaN,BROOKLYN,11217.0,0,0,0,0,0,0,0,0,Sedan,Passing Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified
200,2022-01-02 19:50:00,2022-01-02,19:50,4493137,40.696030,-73.943535,"{'latitude': '40.69603', 'longitude': '-73.943...",Throop Avenue,MYRTLE AVENUE,NaN,Brooklyn,11206,0,0,0,0,0,0,0,0,Motorcycle,Driver Inexperience,Sedan,Unspecified
201,2022-01-02 14:57:00,2022-01-02,14:57,4492828,40.886610,-73.907040,"{'latitude': '40.88661', 'longitude': '-73.907...",WEST 236 STREET,NaN,RIVERDALE AVENUE,The Bronx,10463,0,0,0,0,0,0,0,0,Sedan,Passing or Lane Usage Improper,Sedan,Other Vehicular
203,2022-01-02 00:47:00,2022-01-02,0:47,4493781,40.670994,-73.954960,"{'latitude': '40.670994', 'longitude': '-73.95...",BEDFORD AVENUE,NaN,NaN,Brooklyn,11216,2,0,0,0,2,0,0,0,Bike,Driver Inattention/Distraction,Sedan,Unspecified
204,2022-01-02 01:55:00,2022-01-02,1:55,4494198,40.771305,-73.876640,"{'latitude': '40.771305', 'longitude': '-73.87...",GRAND CENTRAL PKWY,NaN,NaN,Queens,11369,1,0,0,0,0,0,1,0,Sedan,Pavement Slippery,Sedan,Unspecified
206,2022-01-02 13:30:00,2022-01-02,13:30,4494571,40.785114,-73.974960,"{'latitude': '40.785114', 'longitude': '-73.97...",Firefighter Ruben Correa Street,147 WEST 83 STREET,NaN,MANHATTAN,10024.0,0,0,0,0,0,0,0,0,Sedan,Backing Unsafely,Station Wagon/Sport Utility Vehicle,Unspecified
207,2022-01-02 20:15:00,2022-01-02,20:15,4495123,40.605423,-74.120865,"{'latitude': '40.605423', 'longitude': '-74.12...",Manor Road,876 MANOR ROAD,NaN,Staten Island,10314,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified


In [38]:
filtered_df = df[df['zip_code'].isna() & df['latitude'].notnull() & df['borough'].isna()]


In [ ]:
filtered_df = df[df['borough'].isna() & df['latitude'].notnull()]


In [44]:
filtered_df = df[df['borough'].isna() & df['latitude'].notnull()]


In [63]:
filtered_df.head(5)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
1,2022-01-01 05:17:00,2022-01-01,5:17,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,1:30,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.91799,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,520 EAST 137 STREET,NaN,BRONX,10454.0,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
6,2022-01-01 02:53:00,2022-01-01,2:53,4491586,40.646034,-73.99678,"{'latitude': '40.646034', 'longitude': '-73.99...",NaN,850 40 STREET,NaN,BROOKLYN,11232.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified
11,2022-01-01 17:00:00,2022-01-01,17:00,4491660,40.701195,-73.91409,"{'latitude': '40.701195', 'longitude': '-73.91...",WYCKOFF AVENUE,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Sedan,Passing Too Closely,Sedan,Unspecified


In [48]:
test_df = df[df['borough'].isnull()][:5]

In [49]:
len(test_df)

5

In [55]:
# Filter the original DataFrame to get the first 100 rows with missing 'on_street_name'
test_df = df[df['borough'].notnull()].head(5).copy()

# Now 'test_df' contains the first 100 rows with missing 'on_street_name'
test_df

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.917990,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,520 EAST 137 STREET,NaN,BRONX,10454.0,0,0,0,0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
5,2022-01-01 02:09:00,2022-01-01,2:09,4491064,40.714996,-73.810220,"{'latitude': '40.714996', 'longitude': '-73.81...",150 STREET,NaN,COOLIDGE AVENUE,QUEENS,11435.0,0,0,0,0,0,0,0,0,Sedan,Following Too Closely,Sedan,Unspecified
6,2022-01-01 02:53:00,2022-01-01,2:53,4491586,40.646034,-73.996780,"{'latitude': '40.646034', 'longitude': '-73.99...",NaN,850 40 STREET,NaN,BROOKLYN,11232.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified
7,2022-01-01 17:02:00,2022-01-01,17:02,4491430,40.712360,-73.993750,"{'latitude': '40.71236', 'longitude': '-73.993...",MADISON STREET,NaN,MECHANICS ALLEY,MANHATTAN,10002.0,0,0,0,0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Traffic Control Disregarded,Sedan,Unspecified
8,2022-01-01 03:46:00,2022-01-01,3:46,4491096,40.865795,-73.920006,"{'latitude': '40.865795', 'longitude': '-73.92...",WEST 207 STREET,NaN,SHERMAN AVENUE,MANHATTAN,10034.0,0,0,0,0,0,0,0,0,Taxi,Passing Too Closely,Sedan,Unspecified
